In [ ]:
# %load_ext autoreload
# %autoreload 2

import torch

from data import data_helper
from data.data_helper import available_datasets
from models import model_factory
from optimizer.optimizer_helper import get_optim_and_scheduler
from torch.nn import functional as F
from torch import nn

from utils.Logger import Logger

import torchvision
import matplotlib.pyplot as plt

from train_jigsaw import Trainer
from utils import vis

class Container():
    pass

args = Container()
args.batch_size = 128
args.n_classes = 7
args.learning_rate = 0.001
args.epochs = 30
args.network = "caffenet"
args.val_size = 0.1
args.tf_logger = True
args.folder_name = "PACS_alexnet_2" # odd_one_out
args.jigsaw_n_classes = 31

args.train_all = True
# args.jig_weight = 0.9
# args.bias_whole_image = 0.8
args.TTA = False
args.suffix = "nesterov"
args.image_size = 225
args.nesterov = True

args.limit_source = None
args.limit_target = None
args.min_scale = 0.8
args.max_scale = 1.0
args.random_horiz_flip = 0
args.jitter = 0
args.tile_random_grayscale = 0.1


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

source = sorted(["photo", "cartoon", "sketch", "art_painting"])
args.classify_only_sane = True

for args.nesterov in [True, False]:
    for args.jig_weight in [0.5, 0.7, 0.9]: 
        for args.bias_whole_image in [0.6, 0.8, 0.9]:
            for k, x in enumerate(source):
                args.source = source[:k]+source[k+1:]
                args.target = x
                args.suffix = "_%gflipP" % (args.random_horiz_flip)
                if args.nesterov:
                    args.suffix += "_nest"
                for i in range(3):
                    print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                      args.target, 
                                                                      args.jigsaw_n_classes,
                                                                      i))
                    trainer = Trainer(args, device)
                    trainer.do_training()


cartoon-photo-sketch to art_painting - 31 jigsaw classes, split 0
Using Caffe AlexNet
Dataset size: train 7150, val 793, test 2048
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/PACS_alexnet_2/cartoon-photo-sketch_to_art_painting/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0.5_TAll_bias0.6_classifyOnlySane__0flipP_nest_631
New epoch - lr: 0.001
0/55 of epoch 1/30 jigsaw : 3.521, class : 1.967 - acc jigsaw : 3.12, class : 16.41 [bs:128]
30/55 of epoch 1/30 jigsaw : 1.392, class : 0.651 - acc jigsaw : 64.06, class : 65.62 [bs:128]
Accuracies on val: jigsaw : 99.87, class : 86.89
Accuracies on test: jigsaw : 99.76, class : 58.11
New epoch - lr: 0.001
0/55 of epoch 2/30 jigsaw : 1.712, class : 0.370 - acc jigsaw : 57.03, class : 66.41 [bs:128]
30/55 of epoch 2/30 jigsaw : 1.378, class : 0.335 - acc jigsaw : 64.06, class : 77.34 [bs:128]
Accuracies on val: jigsaw : 99.87, class : 88.15
Accuracies on test: jigsaw : 99.41, class : 57.76
New epoch - lr: 0.001

30/55 of epoch 26/30 jigsaw : 0.240, class : 0.011 - acc jigsaw : 92.19, class : 76.56 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 93.44
Accuracies on test: jigsaw : 99.12, class : 67.38
New epoch - lr: 0.0001
0/55 of epoch 27/30 jigsaw : 0.292, class : 0.017 - acc jigsaw : 92.97, class : 76.56 [bs:128]
30/55 of epoch 27/30 jigsaw : 0.243, class : 0.004 - acc jigsaw : 92.97, class : 73.44 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 94.07
Accuracies on test: jigsaw : 98.97, class : 68.12
New epoch - lr: 0.0001
0/55 of epoch 28/30 jigsaw : 0.345, class : 0.034 - acc jigsaw : 89.84, class : 81.25 [bs:128]
30/55 of epoch 28/30 jigsaw : 0.226, class : 0.007 - acc jigsaw : 93.75, class : 76.56 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 93.69
Accuracies on test: jigsaw : 98.93, class : 68.12
New epoch - lr: 0.0001
0/55 of epoch 29/30 jigsaw : 0.232, class : 0.005 - acc jigsaw : 92.19, class : 82.81 [bs:128]
30/55 of epoch 29/30 jigsaw : 0.217, class : 0.005 - acc j

30/55 of epoch 21/30 jigsaw : 0.309, class : 0.011 - acc jigsaw : 87.50, class : 72.66 [bs:128]
Accuracies on val: jigsaw : 99.75, class : 93.19
Accuracies on test: jigsaw : 99.27, class : 67.68
New epoch - lr: 0.001
0/55 of epoch 22/30 jigsaw : 0.446, class : 0.019 - acc jigsaw : 88.28, class : 82.03 [bs:128]
30/55 of epoch 22/30 jigsaw : 0.392, class : 0.011 - acc jigsaw : 85.94, class : 77.34 [bs:128]
Accuracies on val: jigsaw : 99.75, class : 94.07
Accuracies on test: jigsaw : 98.73, class : 68.51
New epoch - lr: 0.001
0/55 of epoch 23/30 jigsaw : 0.438, class : 0.026 - acc jigsaw : 89.06, class : 83.59 [bs:128]
30/55 of epoch 23/30 jigsaw : 0.364, class : 0.039 - acc jigsaw : 89.84, class : 82.81 [bs:128]
Accuracies on val: jigsaw : 99.75, class : 93.95
Accuracies on test: jigsaw : 99.22, class : 65.82
New epoch - lr: 0.001
0/55 of epoch 24/30 jigsaw : 0.334, class : 0.065 - acc jigsaw : 92.19, class : 72.66 [bs:128]
30/55 of epoch 24/30 jigsaw : 0.340, class : 0.037 - acc jigsaw 

30/55 of epoch 16/30 jigsaw : 0.480, class : 0.065 - acc jigsaw : 82.81, class : 75.78 [bs:128]
Accuracies on val: jigsaw : 99.12, class : 94.07
Accuracies on test: jigsaw : 98.68, class : 62.74
New epoch - lr: 0.001
0/55 of epoch 17/30 jigsaw : 0.405, class : 0.121 - acc jigsaw : 85.94, class : 81.25 [bs:128]
30/55 of epoch 17/30 jigsaw : 0.406, class : 0.101 - acc jigsaw : 89.84, class : 76.56 [bs:128]
Accuracies on val: jigsaw : 99.87, class : 93.82
Accuracies on test: jigsaw : 98.88, class : 64.65
New epoch - lr: 0.001
0/55 of epoch 18/30 jigsaw : 0.368, class : 0.033 - acc jigsaw : 91.41, class : 74.22 [bs:128]
30/55 of epoch 18/30 jigsaw : 0.490, class : 0.178 - acc jigsaw : 88.28, class : 74.22 [bs:128]
Accuracies on val: jigsaw : 99.37, class : 93.82
Accuracies on test: jigsaw : 98.88, class : 65.38
New epoch - lr: 0.001
0/55 of epoch 19/30 jigsaw : 0.378, class : 0.058 - acc jigsaw : 89.84, class : 78.12 [bs:128]
30/55 of epoch 19/30 jigsaw : 0.408, class : 0.056 - acc jigsaw 

30/53 of epoch 11/30 jigsaw : 0.776, class : 0.128 - acc jigsaw : 75.00, class : 75.78 [bs:128]
Accuracies on val: jigsaw : 99.48, class : 91.09
Accuracies on test: jigsaw : 97.74, class : 66.94
New epoch - lr: 0.001
0/53 of epoch 12/30 jigsaw : 0.724, class : 0.109 - acc jigsaw : 79.69, class : 78.12 [bs:128]
30/53 of epoch 12/30 jigsaw : 0.693, class : 0.130 - acc jigsaw : 79.69, class : 75.78 [bs:128]
Accuracies on val: jigsaw : 99.61, class : 90.56
Accuracies on test: jigsaw : 97.74, class : 64.55
New epoch - lr: 0.001
0/53 of epoch 13/30 jigsaw : 0.734, class : 0.131 - acc jigsaw : 79.69, class : 78.12 [bs:128]
30/53 of epoch 13/30 jigsaw : 0.692, class : 0.042 - acc jigsaw : 79.69, class : 74.22 [bs:128]
Accuracies on val: jigsaw : 99.87, class : 91.35
Accuracies on test: jigsaw : 98.21, class : 69.16
New epoch - lr: 0.001
0/53 of epoch 14/30 jigsaw : 0.655, class : 0.185 - acc jigsaw : 80.47, class : 76.56 [bs:128]
30/53 of epoch 14/30 jigsaw : 0.617, class : 0.185 - acc jigsaw 

...

0/42 of epoch 1/30 jigsaw : 3.521, class : 1.967 - acc jigsaw : 2.34, class : 11.72 [bs:128]
30/42 of epoch 1/30 jigsaw : 0.856, class : 0.630 - acc jigsaw : 82.03, class : 72.66 [bs:128]
Accuracies on val: jigsaw : 99.83, class : 88.60
Accuracies on test: jigsaw : 100.00, class : 54.59
New epoch - lr: 0.001
0/42 of epoch 2/30 jigsaw : 1.093, class : 0.449 - acc jigsaw : 73.44, class : 78.12 [bs:128]
30/42 of epoch 2/30 jigsaw : 0.704, class : 0.267 - acc jigsaw : 85.16, class : 87.50 [bs:128]
Accuracies on val: jigsaw : 99.83, class : 91.24
Accuracies on test: jigsaw : 100.00, class : 57.39
New epoch - lr: 0.001
0/42 of epoch 3/30 jigsaw : 0.670, class : 0.207 - acc jigsaw : 82.81, class : 85.16 [bs:128]
30/42 of epoch 3/30 jigsaw : 0.704, class : 0.217 - acc jigsaw : 81.25, class : 84.38 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 89.92
Accuracies on test: jigsaw : 100.00, class : 57.88
New epoch - lr: 0.001
0/42 of epoch 4/30 jigsaw : 0.812, class : 0.282 - acc jigsaw : 78.

30/42 of epoch 27/30 jigsaw : 0.408, class : 0.033 - acc jigsaw : 87.50, class : 89.84 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 91.57
Accuracies on test: jigsaw : 97.79, class : 64.14
New epoch - lr: 0.0001
0/42 of epoch 28/30 jigsaw : 0.266, class : 0.002 - acc jigsaw : 92.97, class : 92.97 [bs:128]
30/42 of epoch 28/30 jigsaw : 0.377, class : 0.049 - acc jigsaw : 89.06, class : 93.75 [bs:128]
Accuracies on val: jigsaw : 99.83, class : 90.74
Accuracies on test: jigsaw : 97.58, class : 64.77
New epoch - lr: 0.0001
0/42 of epoch 29/30 jigsaw : 0.250, class : 0.018 - acc jigsaw : 92.97, class : 92.19 [bs:128]
30/42 of epoch 29/30 jigsaw : 0.301, class : 0.020 - acc jigsaw : 89.84, class : 88.28 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 90.91
Accuracies on test: jigsaw : 97.99, class : 63.99
New epoch - lr: 0.0001
0/42 of epoch 30/30 jigsaw : 0.194, class : 0.016 - acc jigsaw : 94.53, class : 92.19 [bs:128]
30/42 of epoch 30/30 jigsaw : 0.318, class : 0.030 - acc ji

30/42 of epoch 22/30 jigsaw : 0.315, class : 0.016 - acc jigsaw : 92.97, class : 91.41 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 91.07
Accuracies on test: jigsaw : 98.83, class : 64.16
New epoch - lr: 0.001
0/42 of epoch 23/30 jigsaw : 0.499, class : 0.023 - acc jigsaw : 83.59, class : 92.97 [bs:128]
30/42 of epoch 23/30 jigsaw : 0.360, class : 0.031 - acc jigsaw : 87.50, class : 87.50 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 89.75
Accuracies on test: jigsaw : 98.75, class : 63.99
New epoch - lr: 0.001
0/42 of epoch 24/30 jigsaw : 0.339, class : 0.065 - acc jigsaw : 88.28, class : 91.41 [bs:128]
30/42 of epoch 24/30 jigsaw : 0.447, class : 0.033 - acc jigsaw : 85.94, class : 83.59 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 90.41
Accuracies on test: jigsaw : 98.88, class : 63.25
New epoch - lr: 0.0001
0/42 of epoch 25/30 jigsaw : 0.386, class : 0.044 - acc jigsaw : 87.50, class : 86.72 [bs:128]
30/42 of epoch 25/30 jigsaw : 0.338, class : 0.023 - acc jig

30/55 of epoch 17/30 jigsaw : 0.221, class : 0.046 - acc jigsaw : 93.75, class : 96.09 [bs:128]
Accuracies on val: jigsaw : 99.87, class : 93.19
Accuracies on test: jigsaw : 99.71, class : 65.33
New epoch - lr: 0.001
0/55 of epoch 18/30 jigsaw : 0.260, class : 0.012 - acc jigsaw : 92.97, class : 92.97 [bs:128]
30/55 of epoch 18/30 jigsaw : 0.223, class : 0.056 - acc jigsaw : 92.97, class : 93.75 [bs:128]
Accuracies on val: jigsaw : 99.87, class : 93.69
Accuracies on test: jigsaw : 99.41, class : 66.46
New epoch - lr: 0.001
0/55 of epoch 19/30 jigsaw : 0.346, class : 0.046 - acc jigsaw : 91.41, class : 92.19 [bs:128]
30/55 of epoch 19/30 jigsaw : 0.160, class : 0.065 - acc jigsaw : 96.09, class : 93.75 [bs:128]
Accuracies on val: jigsaw : 99.87, class : 93.19
Accuracies on test: jigsaw : 99.76, class : 66.16
New epoch - lr: 0.001
0/55 of epoch 20/30 jigsaw : 0.213, class : 0.013 - acc jigsaw : 92.97, class : 94.53 [bs:128]
30/55 of epoch 20/30 jigsaw : 0.172, class : 0.053 - acc jigsaw 

30/55 of epoch 12/30 jigsaw : 0.384, class : 0.117 - acc jigsaw : 90.62, class : 92.97 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 93.69
Accuracies on test: jigsaw : 99.90, class : 63.87
New epoch - lr: 0.001
0/55 of epoch 13/30 jigsaw : 0.287, class : 0.037 - acc jigsaw : 91.41, class : 96.09 [bs:128]
30/55 of epoch 13/30 jigsaw : 0.295, class : 0.053 - acc jigsaw : 91.41, class : 93.75 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 92.81
Accuracies on test: jigsaw : 99.80, class : 65.92
New epoch - lr: 0.001
0/55 of epoch 14/30 jigsaw : 0.362, class : 0.049 - acc jigsaw : 89.84, class : 89.84 [bs:128]
30/55 of epoch 14/30 jigsaw : 0.252, class : 0.061 - acc jigsaw : 92.97, class : 94.53 [bs:128]
Accuracies on val: jigsaw : 99.87, class : 93.57
Accuracies on test: jigsaw : 99.71, class : 63.57
New epoch - lr: 0.001
0/55 of epoch 15/30 jigsaw : 0.252, class : 0.026 - acc jigsaw : 92.19, class : 93.75 [bs:128]
30/55 of epoch 15/30 jigsaw : 0.222, class : 0.030 - acc jigsa

In [ ]:
source = sorted(["photo", "cartoon", "sketch", "art_painting"])
args.epochs = 30
args.jigsaw_n_classes = 31
for args.bias_whole_image in [None, 0.1, 0.3, 0.5]:
    for args.classify_only_sane in [True, False]:
        print("===============================\n")
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()

In [ ]:
source = sorted(["photo", "cartoon", "sketch", "art_painting"])
for args.jigsaw_n_classes in [1,3,5,10,20]:
    args.epochs = int((80/95)*args.jigsaw_n_classes+15)
    for k, x in enumerate(source):
        args.source = source[:k]+source[k+1:]
        args.target = x
        for i in range(3):
            print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                              args.target, 
                                                              args.jigsaw_n_classes,
                                                              i))
            trainer = Trainer(args, device)
            trainer.do_training()

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % ("-".join(args.source),
                                                            args.target,args.epochs, args.jig_weight))

In [ ]:
fig, ax1 = plt.subplots()
for k,v in logger.losses.items():
    ax1.plot(v, label=k)
    l = len(v)
updates = l / len(logger.val_acc)
print(updates)
plt.legend()
ax2 = ax1.twinx()
ax2.plot(range(0,l,int(updates)), logger.val_acc, label="Test acc", c='g')
plt.legend()
plt.title("%s->%s eps:%d jigweight:%.2f" % (str(source),target,epochs, jig_weight))
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
for k,v in logger.losses.items():
    ax1.plot(v, label=k)
    l = len(v)
updates = l / len(logger.val_acc)
print(updates)
plt.legend()
ax2 = ax1.twinx()
ax2.plot(range(0,l,int(updates)), logger.val_acc, label="Test acc", c='g')
plt.legend()
plt.show()

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    return inp

conv1 = models.alexnet(pretrained=True).features[0] #model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
import numpy as np

conv1 = model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
plt.plot(memory["train"], label="train")
plt.plot(memory["val"], label="val")
plt.show()

In [ ]:



# iter_c = iter(train_datasets)

# for x in range(5):
#     tmp = next(iter_c)
#     image = to_plt(tmp[0])
#     plt.imshow(image)
#     plt.show()

In [ ]:
from data.data_helper import get_val_dataloader, get_train_dataloader
from os.path import join, dirname
# from data.JigsawLoader import JigsawTestDataset
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision

loader = get_val_dataloader("photo",31,batch_size=10,multi=False)
# loader, _ = get_train_dataloader(["photo"],31)

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    return inp

# dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)
# test = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, drop_last=False)
iter_c = iter(loader)
(tmp, v, c), d = next(iter_c)

for x in range(tmp.shape[0]):
#     image = tmp[0, x]
    image = torchvision.utils.make_grid(tmp[x],1,normalize=True)
    plt.imshow(to_plt(image))
    plt.show()

    
# print(v.max(), v.min())

In [ ]:
tmp.max(), tmp.min()

In [ ]:
from data.JigsawLoader import JigsawDataset
from PIL import Image
import torchvision.transforms as transforms

class JigsawTestDataset(JigsawDataset):
    def __init__(self, *args, **xargs):
        super().__init__(*args, **xargs)
        self._augment_tile = transforms.Compose([
#             transforms.RandomCrop(64),
            transforms.Resize((75, 75), Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
    def __getitem__(self, index):
        framename = self.data_path + '/' + self.names[index]
        img = Image.open(framename).convert('RGB')
        img = self._image_transformer(img)

        w = float(img.size[0]) / self.grid_size
        n_grids = self.grid_size ** 2
        tiles = [None] * n_grids
        for n in range(n_grids):
            y = int(n / self.grid_size)
            x = n % self.grid_size
            tile = img.crop([x * w, y * w, (x + 1) * w, (y + 1) * w])
            tile = self._augment_tile(tile)
            tiles[n] = tile

        data = torch.stack(tiles, 0)
        return self.returnFunc(data), 0, int(self.labels[index])


In [ ]:
dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)

In [ ]:
sorted([np.random.randint(9) for x in range(50)])

In [ ]:
import numpy as np

In [ ]:
perms = np.load("permutations_31.npy")
perms

In [ ]:
def stuff():
    print("done")

a = {"ciao":0}
try:
    a["cigao"]
except KeyError:
    stuff()

In [ ]:
a = list(range(10))
a[:None]